In [1]:
from tqdm.notebook import tqdm
import random
import torch
from torch import nn
from torch import optim
import numpy as np

In [2]:
data = np.load("eu_d.npy", allow_pickle=True)

In [3]:
data.shape

(456456, 2)

In [4]:
class ResBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim),
            nn.LeakyReLU(),
        )
        
    def forward(self, x):
        return x + self.net(x)

In [5]:
net = nn.Sequential(
    nn.Linear(2,16),
    nn.LeakyReLU(),
    nn.Linear(16,16),
    nn.LeakyReLU(),
#     ResBlock(128),
#     ResBlock(128),
#     ResBlock(128),
#     ResBlock(128),
#     ResBlock(128),
#     ResBlock(128),
    nn.Linear(16,16),
    nn.LeakyReLU(),
    nn.Linear(16,1),
)

In [6]:
optimizer = optim.SGD(net.parameters(), lr=0.0001)
criterion = nn.MSELoss()

In [7]:
train_loader = torch.utils.data.DataLoader(data[:-5000], batch_size=1000)

In [8]:
val_loader = torch.utils.data.DataLoader(data[-5000:], batch_size=1000)

In [9]:
for epoch in range(1000):
    net.train()
    for eu_d in (bar := tqdm(train_loader)):
#         eu_d = torch.rand_like(eu_d) * 10
        net.zero_grad()
        sqdiff = torch.tensor(eu_d[:, 0] - eu_d[:, 1])[:, None].square()
        pred = net(eu_d)
        loss = criterion(pred, sqdiff)
        loss.backward()
        optimizer.step()
        bar.set_postfix({"err": np.sqrt(loss.item())})
    net.eval()
    err = []
    with torch.no_grad():
        for eu_d in val_loader:
#             eu_d = torch.rand_like(eu_d) * 10
            sqdiff = torch.tensor(eu_d[:, 0] - eu_d[:, 1])[:, None].square()
            pred = net(eu_d)
            err += torch.abs((pred - sqdiff) / sqdiff).tolist()
        print(torch.cat([sqdiff, pred], dim=1)[:10, :])
    print(epoch, np.median(err), np.mean(err))

  0%|          | 0/452 [00:00<?, ?it/s]

<ipython-input-9-31c122b01119>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sqdiff = torch.tensor(eu_d[:, 0] - eu_d[:, 1])[:, None].square()


tensor([[4.5552e+00, 3.1450e-01],
        [5.5520e-01, 2.5995e-01],
        [1.1214e-04, 4.7396e-01],
        [2.6649e-01, 2.7007e-01],
        [9.8741e-04, 1.6460e-01],
        [7.7605e-01, 3.3333e-01],
        [1.2910e-02, 2.1144e-01],
        [5.6792e-03, 3.3188e-01],
        [1.5759e+00, 3.2753e-01],
        [4.4123e-01, 2.6940e-01]])
0 0.8908132612705231 37755.59689542445


<ipython-input-9-31c122b01119>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sqdiff = torch.tensor(eu_d[:, 0] - eu_d[:, 1])[:, None].square()


  0%|          | 0/452 [00:00<?, ?it/s]

...

tensor([[ 4.5552e+00,  4.7401e+00],
        [ 5.5520e-01,  5.6168e-01],
        [ 1.1214e-04, -4.8921e-02],
        [ 2.6649e-01,  3.1449e-01],
        [ 9.8741e-04,  1.1447e-03],
        [ 7.7605e-01,  7.3262e-01],
        [ 1.2910e-02,  4.1063e-02],
        [ 5.6792e-03, -9.8891e-02],
        [ 1.5759e+00,  1.6645e+00],
        [ 4.4123e-01,  4.1674e-01]])
997 0.12645737081766129 4232.213050413121


  0%|          | 0/452 [00:00<?, ?it/s]

tensor([[ 4.5552e+00,  4.7403e+00],
        [ 5.5520e-01,  5.6167e-01],
        [ 1.1214e-04, -4.8916e-02],
        [ 2.6649e-01,  3.1447e-01],
        [ 9.8741e-04,  1.0865e-03],
        [ 7.7605e-01,  7.3259e-01],
        [ 1.2910e-02,  4.1065e-02],
        [ 5.6792e-03, -9.8883e-02],
        [ 1.5759e+00,  1.6644e+00],
        [ 4.4123e-01,  4.1672e-01]])
998 0.12623411417007446 4231.675096756061


  0%|          | 0/452 [00:00<?, ?it/s]

tensor([[ 4.5552e+00,  4.7407e+00],
        [ 5.5520e-01,  5.6165e-01],
        [ 1.1214e-04, -4.8898e-02],
        [ 2.6649e-01,  3.1446e-01],
        [ 9.8741e-04,  1.0276e-03],
        [ 7.7605e-01,  7.3256e-01],
        [ 1.2910e-02,  4.1068e-02],
        [ 5.6792e-03, -9.8866e-02],
        [ 1.5759e+00,  1.6644e+00],
        [ 4.4123e-01,  4.1670e-01]])
999 0.12590258568525314 4230.27357420061


In [439]:
print(net(torch.tensor([[0., 0.],
                        [0., 1],
                        [0., 2],
                        [0., 3],
                        [0., 4],
                        [0., 5],
                        [0., 6],
                        [0., 7],
                        [0., 8],
                        [0., 9]])))

tensor([[ 0.8370],
        [ 1.3064],
        [ 3.7761],
        [ 8.9825],
        [17.3631],
        [27.4014],
        [37.5714],
        [47.5369],
        [57.0769],
        [66.3386]], grad_fn=<AddmmBackward>)


In [112]:
net.eval()
err = []
with torch.no_grad():
    for eu_d in (bar := tqdm(data[-5000:, :])):
        sqdiff = torch.tensor(eu_d[0] - eu_d[1]).square()
        pred = net(torch.tensor(eu_d))
        err.append(np.abs(pred.item() - sqdiff.item()) / sqdiff.item())
        print(pred.item(), sqdiff.item())

  0%|          | 0/10000 [00:00<?, ?it/s]

0.4058016538619995 1.343533992767334
24.172975540161133 28.480993270874023
3.702401876449585 3.4859619140625
0.46427279710769653 0.11884527653455734
0.4227063059806824 1.0068390369415283
0.46118849515914917 0.008778247982263565
0.4758131504058838 0.03400048986077309
0.4633966088294983 0.10225093364715576
0.46653103828430176 0.006437885574996471
0.41779059171676636 0.6680024266242981
0.4586895704269409 0.09229059517383575
0.4427664875984192 0.5741840600967407
0.4328789710998535 0.302799791097641
0.6527474522590637 0.44908860325813293
0.42768287658691406 0.7422394156455994
0.4410039782524109 0.10152947157621384
0.4796168804168701 0.016825322061777115
0.465549111366272 0.2962101399898529
0.4572637677192688 0.03287649154663086
0.47245150804519653 0.0564974844455719
10.757575035095215 6.9920334815979
0.4306986927986145 0.3750700354576111
0.45801401138305664 0.10852419584989548
0.4778231978416443 0.007147960364818573
0.43711429834365845 0.5711864233016968
0.4567188620567322 0.273633509874343

In [113]:
np.median(err)

0.6662024000679129

In [222]:
net.eval()

Sequential(
  (0): Linear(in_features=2, out_features=4, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Linear(in_features=4, out_features=8, bias=True)
  (4): LeakyReLU(negative_slope=0.01)
  (5): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Linear(in_features=8, out_features=8, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): Linear(in_features=8, out_features=8, bias=True)
  (10): LeakyReLU(negative_slope=0.01)
  (11): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): Linear(in_features=8, out_features=4, bias=True)
  (13): LeakyReLU(negative_slope=0.01)
  (14): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (15): Linear(in_features=4, out_features=2, bias=True)
  (16): LeakyReLU(negative

In [252]:
net(torch.tensor([[0., 0.],[0., .5]]))

tensor([[0.1162],
        [0.1162]], grad_fn=<AddmmBackward>)